## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Вывод

Есть пропуски в двух столбцах: "days_employed" и "total_income". Общий трудовой стаж мог быть пропущен, потому что на момент заполнения анкеты, клиент мог не знать его. Ежемесячный доход мог быть пропущен, так как эта информация напряпую влияет на конечное решение о выдоваемом кредите. Поскольку, перед нами стоит задача, где в анализе не будет учавствовать общий трудовой стаж - этот столбец можно оставить таким, какой он есть, но сообщить команде о пропусках. А вот столбец "total_income" надо обработать на наличие пропусков. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df_grouped = df.groupby('income_type')['total_income'].median()
df_grouped

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [3]:
total_income_median = df['total_income'].median()
total_income_median

145017.93753253992

In [4]:
df['total_income'] = df['total_income'].fillna(total_income_median)
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Вывод

Для заполнения пропусков по столбцу "total_income" взяли число равное медиане по всему столбцу. Так как медиада более точно охариктеризует средний доход по выборке. Так же с помощью метода groupby() по столбцу income_type и подсчету медианы по группам, было выявлено, что среди всех групп есть люди с доходом, то есть какая-то определенная группа не с доходом 0р. Можно сделать вывод, что пропуски были не по какой-то группе, а при заполнении данных о клиентах. О них стоит сообщить команде.

In [5]:
df[df['days_employed'].isnull()].head(10)
df[df['days_employed'].isnull()]['income_type'].value_counts(normalize = True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [6]:
df['income_type'].value_counts(normalize = True)

сотрудник          0.516562
компаньон          0.236237
пенсионер          0.179141
госслужащий        0.067782
предприниматель    0.000093
безработный        0.000093
студент            0.000046
в декрете          0.000046
Name: income_type, dtype: float64

### Замена типа данных

In [7]:
convert_dict = { 
                'total_income': int
               } 
 
df = df.astype(convert_dict) 
print(df.dtypes) 

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object


### Вывод

Основная работа будет проводиться со столбцом 'total_income' - его значения мы перевели в целочисленные. Метод astype преобразует серию, в которой нет пропусков. 

### Обработка дубликатов

In [8]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [9]:
df['education'] = df['education'].str.lower()

In [10]:
df['family_status'] = df['family_status'].str.lower()

In [11]:
df['gender'] = df['gender'].str.lower()


In [12]:
df['income_type'] = df['income_type'].str.lower()


In [13]:
df['purpose'] = df['purpose'].str.lower()


In [14]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

### Вывод

Все строки с типом object были переведены в нижний регистр, для более удобной обработки и визуализации данных. Это так же способствовало большему нахождению дубликотов. Дубликаты удалены методом drop_duplicates(), и чтобы не нарушать порядок индексации, так же был вызван метод reset_index(). Так, из таблицы были удалены грубые повторы, которые мешали бы дальнейшему анализу.Дубликаты могли оказаться по ошибке, при переносе данных из источника. О их наличии стоит также сообщить команде. Все строки с типом object были переведены в нижний регистр, для более удобной обработки и визуализации данных.

### Лемматизация

In [15]:
df['purpose'].value_counts().head(10)

свадьба                                  791
на проведение свадьбы                    768
сыграть свадьбу                          765
операции с недвижимостью                 675
покупка коммерческой недвижимости        661
операции с жильем                        652
покупка жилья для сдачи                  651
операции с коммерческой недвижимостью    650
жилье                                    646
покупка жилья                            646
Name: purpose, dtype: int64

In [16]:
df['family_status'].value_counts().head(15)

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [17]:
from pymystem3 import Mystem
m = Mystem() 
def lemmatizator(i):
    lemmas = m.lemmatize(i)
    return lemmas
df['purpose_lemmas'] = df['purpose'].apply(lemmatizator)

In [18]:
def purpose_type(purpose_lemmas):

    if 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'недвижимость' in purpose_lemmas:
        return 'недвижимость'
    if 'квартира' in purpose_lemmas:
        return 'недвижимость' 
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    else:
        return 'без категории'
df['purpose_type'] = df['purpose_lemmas'].apply(purpose_type)
df['purpose_type'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_type, dtype: int64

### Вывод

Выделили 4 главных группы: 
недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
В недвижимость вошли запросы на жилье, квартиру, и на коммерческую недвижимость, а так же операции со собственной недвижимостью и пр. Это самый популярный запрос. Самый не популярный - свадьба. 



### Категоризация данных

In [19]:
df_new = df.loc[:, ('children','family_status', 'total_income', 'purpose_type', 'debt')]
def chilgren_group(children):
    if children ==  0 :
        return 'бездетная'
    if 1 <= children:
        return 'c детьми'
    return 'неизвестно'

df_new['chilgren_group'] = df['children'].apply(chilgren_group)
df_new.head(5)
    
    

,children,family_status,total_income,purpose_type,debt,chilgren_group
0,1,женат / замужем,253875,недвижимость,0,c детьми
1,1,женат / замужем,112080,автомобиль,0,c детьми
2,0,женат / замужем,145885,недвижимость,0,бездетная
3,3,женат / замужем,267628,образование,0,c детьми
4,0,гражданский брак,158616,свадьба,0,бездетная


In [20]:
df['total_income'].describe()

count    2.145400e+04
mean     1.652256e+05
std      9.802102e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450170e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [21]:
def total_income_group(total_income):
    if total_income < 100000:
        return 'низкий'
    if  100000 < total_income < 200000:
        return 'средний'
    if 200000 < total_income < 400000:
        return 'выше среднего'
    return 'высокий'
df_new['total_income_group'] = df['total_income'].apply(total_income_group)

df_new.head(5)

,children,family_status,total_income,purpose_type,debt,chilgren_group,total_income_group
0,1,женат / замужем,253875,недвижимость,0,c детьми,выше среднего
1,1,женат / замужем,112080,автомобиль,0,c детьми,средний
2,0,женат / замужем,145885,недвижимость,0,бездетная,средний
3,3,женат / замужем,267628,образование,0,c детьми,выше среднего
4,0,гражданский брак,158616,свадьба,0,бездетная,средний


### Вывод

Две новые категории: total_income_group и chilgren_group.
В total_income_group данные были разбиты по группам, в зависимоти от уровня дохода: низкий - до 100000 рублей, средний - от 100000 до 200000 рублей, выше среднего - от 200000 до 400000 рублей, и высокий - от 400000 и выше. 
В chilgren_group выделены три группы: 0 детей - бездетная, остальные с детьми, так как для дальнейшего анализа нам необходимо наличие или отсутвие детей в семье, при оформлении кредита. 


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
df_new.pivot_table(index = 'chilgren_group', values = 'debt', aggfunc = ('count','mean'))


,count,mean
chilgren_group,,
c детьми,7316.0,0.092537
бездетная,14091.0,0.075438
неизвестно,47.0,0.021277


### Вывод

Просрочка кредита в срок у заемщиков с детьми в 9% случаев. Бездетные не выплачивают в срок в 7%. Люди, чей статус неизвестен по наличию детей, просрочили всего в 2% случаев. Надо передать эту ифнормацию команде, для уточнения данных, чтобы исключить искажения результатов. Так, наличие детей  увеличивает риск не погашения кредита в срок на ~2%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
df_new.pivot_table(index='family_status',values='debt',aggfunc=('count','mean'))


,count,mean
family_status,,
в разводе,1195.0,0.071130
вдовец / вдова,959.0,0.065693
гражданский брак,4151.0,0.093471
женат / замужем,12339.0,0.075452
не женат / не замужем,2810.0,0.097509


### Вывод

Процент просрочки по кредитам: 
"не женат / не замужем" ~9,7%; 
"гражданский брак" ~9,3%;
"женат / замужем" ~7,5%;
"в разводе" ~ 7,1%;
"вдовец / вдова	" ~6,5%.
Группы с самым большим риском по не возрату кредита в срок: "не женат / не замужем" и "гражданский брак". Люди, которые состояли в официально оформленном браке или состоят в нем на момент взятия кредита, возвращают его вовремя на ~2-3% чаще, чем люди в статусе "не женат / не замужем" и "гражданский брак".



- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
df_new.pivot_table(index='total_income_group',values='debt',aggfunc=('count','mean'))


,count,mean
total_income_group,,
высокий,528.0,0.058712
выше среднего,4538.0,0.072058
низкий,4463.0,0.079319
средний,11925.0,0.086289


### Вывод

Процентр просрочки по кредитам согласно уровню дохода:
средний ~8,6%; 
низкий ~7,9%;
выше среднего ~7,2%;
высокий ~5,8%.
Самый низкий риск у группы с уровнем дохода "высокий". А вот группа людей, чей доход "средний" и низкий" с большей вероятностью не погасят кредит в срок. Зависимость между уровнем дохода и возвратом кредита в срок есть.

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
df_new.pivot_table(index='purpose_type',values='debt',aggfunc=('count','mean'))


,count,mean
purpose_type,,
автомобиль,4306.0,0.093590
недвижимость,10811.0,0.072334
образование,4013.0,0.092200
свадьба,2324.0,0.080034


### Вывод

Цели заема влияют на погашения кредита в срок: 
автомобиль ~ 9.3% что кредит не будет закрыт вовремя;
образование ~ 9.2%;
свадьба ~ 8.0%;
недвижимость ~ 7.2%.
В зависимости от цели кредита риски варьируются в районе 2%.


In [26]:
df_new.pivot_table(index='family_status',values='debt',aggfunc=('count','mean'))

,count,mean
family_status,,
в разводе,1195.0,0.071130
вдовец / вдова,959.0,0.065693
гражданский брак,4151.0,0.093471
женат / замужем,12339.0,0.075452
не женат / не замужем,2810.0,0.097509


В результате исполнения одной строки кода мы наглядно видим, какие категории возвращают кредит чаще/реже.


### Шаг 4. Общий вывод

Семейное положение и наличие детей влияет на факт возврата кредита в срок. Наличие детей увеличивает риск просрочки на 2%. А статус семейного положения важен в контексте: был ли человек когда-либо официально расписан или нет, так как выборка людей, которая причислила себя к группам "в гражданском браке" и "не женат/ не замужем", не возвращают кредит в срок на  ~2-3% чаще, чем остальные группы, состоящие когда либо в официальном браке. Так же на успешную выплату по кредиту влияет сама цель кредита и ежемесячный доход заемщика. Цели с самым высоким риском просрочки: "автомобиль" и "образование, а самый низкий риск у неджимости. Люди с высоким уровнем дохода(более 400000 рублей в месяц) реже всего не возвращают кредит в срок, чем люди с низким (менее 100 000р) и средним (от 100000р до 200000р).  

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.